In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit # pip install verstack
from nltk.corpus import stopwords 
#import nltk
#nltk.download('stopwords')
# Load the training data
train_data = pd.read_csv("train.csv")
def hash_to_list(x):
    return [ (i.strip())[1:-1] for i in x[1:-1].split(",")]
def extract_part_of_link(x):
    return [i.split('/')[2] if i!="" else "" for i in x]
train_data["hashtags"] = train_data["hashtags"].apply(hash_to_list)
train_data["urls"] = train_data["urls"].apply(hash_to_list)
train_data["mentions"] = train_data["mentions"].apply(hash_to_list)
#train_data["urls_site"] = train_data["urls"].apply(extract_part_of_link)
# Here we split our training data into trainig and testing set. This way we can estimate the evaluation of our model without uploading to Kaggle and avoid overfitting over our evaluation dataset.
# scsplit method is used in order to split our regression data in a stratisfied way and keep a similar distribution of retweet counts between the two sets
X_train, X_test, y_train, y_test = scsplit(train_data, train_data['retweets_count'], stratify=train_data['retweets_count'], train_size=0.7, test_size=0.3)

# We remove the actual number of retweets from our features since it is the value that we are trying to predict
X_train = X_train.drop(['retweets_count'], axis=1)
X_test = X_test.drop(['retweets_count'], axis=1)

# You can examine the available features using X_train.head()
# X_train.head()
t = train_data.iloc[20]["urls"]

2022-12-04 14:19:11.212332: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-04 14:19:11.212362: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
X_train.dtypes


text               object
favorites_count     int64
followers_count     int64
statuses_count      int64
friends_count       int64
mentions           object
urls               object
verified            int64
hashtags           object
timestamp           int64
TweetID             int64
dtype: object

In [3]:
#c = train_data.groupby("urls").size().reset_index(name='counts')
from collections import Counter
columns = ["urls", "mentions", "hashtags"]
for c in columns:
    d = dict()
    for urls in train_data[c]:
        if urls[0] == "":
            continue
        for url in urls:
            if url in d.keys():
                d[url]+=1
            else:
                d[url] = 1
    res = Counter(d.values())
    print(f"Statistic for column \"{c}\" is : {res}")


Statistic for column "urls" is : Counter({1: 184815, 2: 1043, 3: 51, 4: 20, 5: 8, 7: 4, 8: 3, 9: 2, 6: 2, 18: 1, 11: 1, 10: 1})
Statistic for column "mentions" is : Counter()
Statistic for column "hashtags" is : Counter({1: 7175, 2: 2108, 3: 704, 4: 444, 5: 258, 6: 207, 7: 151, 8: 106, 9: 98, 10: 61, 11: 56, 12: 42, 13: 36, 14: 35, 15: 30, 17: 27, 16: 27, 18: 25, 24: 23, 21: 20, 20: 19, 23: 16, 22: 15, 27: 14, 30: 14, 25: 14, 33: 13, 19: 12, 26: 10, 32: 9, 40: 9, 53: 8, 39: 8, 31: 8, 41: 8, 28: 8, 59: 7, 29: 7, 43: 6, 61: 6, 46: 6, 48: 6, 47: 6, 37: 5, 57: 5, 35: 5, 45: 5, 88: 4, 42: 4, 49: 4, 52: 4, 70: 4, 242: 3, 34: 3, 54: 3, 36: 3, 63: 3, 83: 3, 130: 3, 62: 3, 75: 3, 44: 3, 458: 2, 97: 2, 93: 2, 51: 2, 112: 2, 133: 2, 205: 2, 255: 2, 79: 2, 84: 2, 131: 2, 60: 2, 153: 2, 76: 2, 119: 2, 106: 2, 248: 2, 65: 2, 58: 2, 180: 2, 87: 2, 55: 2, 38: 2, 2058: 1, 1725: 1, 1032: 1, 1376: 1, 2075: 1, 406: 1, 451: 1, 2063: 1, 827: 1, 1284: 1, 13928: 1, 550: 1, 483: 1, 5629: 1, 238: 1, 230: 1, 310

In [4]:
# We set up an Tfidf Vectorizer that will use the top 100 tokens from the tweets. We also remove stopwords.
# To do that we have to fit our training dataset and then transform both the training and testing dataset. 
vectorizer = TfidfVectorizer(max_features=100, stop_words=stopwords.words('french'))
X_train_text = vectorizer.fit_transform(X_train['text'])
X_test_text = vectorizer.transform(X_test['text'])
print(X_train_text.shape)
train_columns = ["favorites_count","followers_count", "statuses_count", "friends_count", "verified", "timestamp"]
print(X_train.loc[:, X_train.columns != 'text'].to_numpy().shape)
X_train_all = np.hstack((X_train.loc[:, X_train.columns != 'text'].to_numpy(), X_train_text.toarray()))
X_test_all = np.hstack((X_test.loc[:, X_test.columns != 'text'].to_numpy(), X_test_text.toarray()))

(247778, 100)
(247778, 10)


## GradientBoosting

In [5]:
# Now we can train our model. Here we chose a Gradient Boosting Regressor and we set our loss function 
reg = GradientBoostingRegressor()#reg = RandomForestRegressor() #
#reg = LinearRegression()

# We fit our model using the training data
reg.fit(X_train.loc[:,train_columns] , y_train)
# And then we predict the values for our testing set
y_pred = reg.predict(X_test.loc[:,train_columns])
# We want to make sure that all predictions are non-negative integers
y_pred = [int(value) if value >= 0 else 0 for value in y_pred]

print("Prediction error:", mean_absolute_error(y_true=y_test, y_pred=y_pred))



Prediction error: 6.784275503573749


## Catboost

In [6]:
from catboost import CatBoostRegressor, Pool, metrics, cv
from sklearn.metrics import accuracy_score

model = CatBoostRegressor(
    random_seed=42,
)
categorical_features_indices = []

model.fit(
    X_train.loc[:,train_columns], y_train,
    cat_features=categorical_features_indices,
    plot=True
);

y_pred = model.predict(X_test.loc[:,train_columns])
# We want to make sure that all predictions are non-negative integers
y_pred = [int(value) if value >= 0 else 0 for value in y_pred]

print("Prediction error:", mean_absolute_error(y_true=y_test, y_pred=y_pred))


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.097823
0:	learn: 203.6213284	total: 65.7ms	remaining: 1m 5s
1:	learn: 192.3905348	total: 74.7ms	remaining: 37.3s
2:	learn: 181.3726231	total: 84.3ms	remaining: 28s
3:	learn: 172.3443375	total: 93.1ms	remaining: 23.2s
4:	learn: 163.5791659	total: 102ms	remaining: 20.2s
5:	learn: 156.1511649	total: 110ms	remaining: 18.2s
6:	learn: 149.7761524	total: 120ms	remaining: 17s
7:	learn: 143.8549917	total: 129ms	remaining: 16s
8:	learn: 138.3155815	total: 138ms	remaining: 15.2s
9:	learn: 133.4949675	total: 147ms	remaining: 14.5s
10:	learn: 128.9186849	total: 156ms	remaining: 14.1s
11:	learn: 124.6009833	total: 167ms	remaining: 13.7s
12:	learn: 120.7930063	total: 177ms	remaining: 13.4s
13:	learn: 117.3287281	total: 186ms	remaining: 13.1s
14:	learn: 114.0367556	total: 194ms	remaining: 12.8s
15:	learn: 111.0758002	total: 206ms	remaining: 12.7s
16:	learn: 108.4517258	total: 216ms	remaining: 12.5s
17:	learn: 106.0079517	total: 225ms	remaining: 12.3s
18:	learn: 103.8556078	total

165:	learn: 52.5141625	total: 1.58s	remaining: 7.93s
166:	learn: 52.4212975	total: 1.59s	remaining: 7.92s
167:	learn: 52.3105189	total: 1.6s	remaining: 7.92s
168:	learn: 52.2391334	total: 1.61s	remaining: 7.91s
169:	learn: 52.1544060	total: 1.62s	remaining: 7.89s
170:	learn: 52.1123866	total: 1.63s	remaining: 7.88s
171:	learn: 52.0061888	total: 1.63s	remaining: 7.87s
172:	learn: 51.8633372	total: 1.64s	remaining: 7.86s
173:	learn: 51.7292139	total: 1.65s	remaining: 7.85s
174:	learn: 51.5831667	total: 1.66s	remaining: 7.84s
175:	learn: 51.4977884	total: 1.67s	remaining: 7.82s
176:	learn: 51.3645418	total: 1.68s	remaining: 7.82s
177:	learn: 51.3455242	total: 1.69s	remaining: 7.81s
178:	learn: 51.2924055	total: 1.7s	remaining: 7.79s
179:	learn: 51.1686871	total: 1.71s	remaining: 7.78s
180:	learn: 51.1251360	total: 1.72s	remaining: 7.79s
181:	learn: 51.0044139	total: 1.73s	remaining: 7.78s
182:	learn: 50.8677666	total: 1.74s	remaining: 7.76s
183:	learn: 50.7647531	total: 1.75s	remaining: 7

334:	learn: 41.8782737	total: 2.98s	remaining: 5.92s
335:	learn: 41.8578385	total: 2.99s	remaining: 5.91s
336:	learn: 41.8284583	total: 3s	remaining: 5.9s
337:	learn: 41.7435086	total: 3.01s	remaining: 5.89s
338:	learn: 41.6849117	total: 3.02s	remaining: 5.88s
339:	learn: 41.6371469	total: 3.02s	remaining: 5.87s
340:	learn: 41.5821714	total: 3.03s	remaining: 5.86s
341:	learn: 41.5365898	total: 3.04s	remaining: 5.86s
342:	learn: 41.4432290	total: 3.05s	remaining: 5.85s
343:	learn: 41.4001745	total: 3.06s	remaining: 5.83s
344:	learn: 41.3631217	total: 3.07s	remaining: 5.83s
345:	learn: 41.3172117	total: 3.08s	remaining: 5.82s
346:	learn: 41.2486655	total: 3.09s	remaining: 5.81s
347:	learn: 41.2206511	total: 3.1s	remaining: 5.81s
348:	learn: 41.1724324	total: 3.11s	remaining: 5.79s
349:	learn: 41.0880706	total: 3.12s	remaining: 5.79s
350:	learn: 41.0769793	total: 3.13s	remaining: 5.78s
351:	learn: 41.0399924	total: 3.13s	remaining: 5.77s
352:	learn: 41.0144118	total: 3.14s	remaining: 5.76

499:	learn: 36.2068359	total: 4.38s	remaining: 4.38s
500:	learn: 36.1543225	total: 4.39s	remaining: 4.37s
501:	learn: 36.0852131	total: 4.4s	remaining: 4.36s
502:	learn: 36.0625378	total: 4.41s	remaining: 4.36s
503:	learn: 36.0072921	total: 4.42s	remaining: 4.34s
504:	learn: 36.0001741	total: 4.42s	remaining: 4.34s
505:	learn: 35.9799229	total: 4.43s	remaining: 4.33s
506:	learn: 35.9541044	total: 4.44s	remaining: 4.32s
507:	learn: 35.9186037	total: 4.45s	remaining: 4.31s
508:	learn: 35.8689931	total: 4.46s	remaining: 4.3s
509:	learn: 35.8440390	total: 4.46s	remaining: 4.29s
510:	learn: 35.8265075	total: 4.47s	remaining: 4.28s
511:	learn: 35.7887699	total: 4.48s	remaining: 4.27s
512:	learn: 35.7569764	total: 4.49s	remaining: 4.26s
513:	learn: 35.7356999	total: 4.5s	remaining: 4.25s
514:	learn: 35.7082294	total: 4.5s	remaining: 4.24s
515:	learn: 35.6746846	total: 4.51s	remaining: 4.23s
516:	learn: 35.6427809	total: 4.52s	remaining: 4.22s
517:	learn: 35.6143380	total: 4.53s	remaining: 4.2

655:	learn: 32.3725885	total: 5.89s	remaining: 3.09s
656:	learn: 32.3302659	total: 5.9s	remaining: 3.08s
657:	learn: 32.3048027	total: 5.91s	remaining: 3.07s
658:	learn: 32.3001589	total: 5.92s	remaining: 3.06s
659:	learn: 32.2723267	total: 5.92s	remaining: 3.05s
660:	learn: 32.2549109	total: 5.93s	remaining: 3.04s
661:	learn: 32.2377747	total: 5.94s	remaining: 3.03s
662:	learn: 32.2313544	total: 5.95s	remaining: 3.02s
663:	learn: 32.2253039	total: 5.96s	remaining: 3.01s
664:	learn: 32.1926040	total: 5.96s	remaining: 3s
665:	learn: 32.1821855	total: 5.97s	remaining: 3s
666:	learn: 32.1687529	total: 5.98s	remaining: 2.99s
667:	learn: 32.1545902	total: 5.99s	remaining: 2.98s
668:	learn: 32.1441941	total: 6s	remaining: 2.97s
669:	learn: 32.1300544	total: 6s	remaining: 2.96s
670:	learn: 32.1169411	total: 6.01s	remaining: 2.95s
671:	learn: 32.0822557	total: 6.02s	remaining: 2.94s
672:	learn: 32.0737686	total: 6.05s	remaining: 2.94s
673:	learn: 32.0489894	total: 6.07s	remaining: 2.93s
674:	l

813:	learn: 29.4085518	total: 7.3s	remaining: 1.67s
814:	learn: 29.3963874	total: 7.3s	remaining: 1.66s
815:	learn: 29.3913878	total: 7.31s	remaining: 1.65s
816:	learn: 29.3784392	total: 7.32s	remaining: 1.64s
817:	learn: 29.3608472	total: 7.33s	remaining: 1.63s
818:	learn: 29.3343123	total: 7.34s	remaining: 1.62s
819:	learn: 29.3232639	total: 7.35s	remaining: 1.61s
820:	learn: 29.3195017	total: 7.36s	remaining: 1.6s
821:	learn: 29.2916421	total: 7.37s	remaining: 1.6s
822:	learn: 29.2726930	total: 7.38s	remaining: 1.59s
823:	learn: 29.2478096	total: 7.39s	remaining: 1.58s
824:	learn: 29.2404447	total: 7.4s	remaining: 1.57s
825:	learn: 29.2328596	total: 7.41s	remaining: 1.56s
826:	learn: 29.2271038	total: 7.42s	remaining: 1.55s
827:	learn: 29.2108752	total: 7.43s	remaining: 1.54s
828:	learn: 29.1949740	total: 7.44s	remaining: 1.53s
829:	learn: 29.1860492	total: 7.45s	remaining: 1.53s
830:	learn: 29.1803396	total: 7.46s	remaining: 1.52s
831:	learn: 29.1633063	total: 7.47s	remaining: 1.51

976:	learn: 27.0340767	total: 8.87s	remaining: 209ms
977:	learn: 27.0286991	total: 8.88s	remaining: 200ms
978:	learn: 27.0022800	total: 8.89s	remaining: 191ms
979:	learn: 26.9911551	total: 8.89s	remaining: 182ms
980:	learn: 26.9714876	total: 8.9s	remaining: 172ms
981:	learn: 26.9590410	total: 8.91s	remaining: 163ms
982:	learn: 26.9353910	total: 8.92s	remaining: 154ms
983:	learn: 26.9168013	total: 8.93s	remaining: 145ms
984:	learn: 26.9018646	total: 8.94s	remaining: 136ms
985:	learn: 26.8973119	total: 8.95s	remaining: 127ms
986:	learn: 26.8878272	total: 8.96s	remaining: 118ms
987:	learn: 26.8783313	total: 8.97s	remaining: 109ms
988:	learn: 26.8688978	total: 8.97s	remaining: 99.8ms
989:	learn: 26.8595132	total: 8.98s	remaining: 90.7ms
990:	learn: 26.8488052	total: 8.99s	remaining: 81.7ms
991:	learn: 26.8223904	total: 9s	remaining: 72.6ms
992:	learn: 26.8144192	total: 9.01s	remaining: 63.5ms
993:	learn: 26.8027450	total: 9.02s	remaining: 54.4ms
994:	learn: 26.7992904	total: 9.03s	remainin

## Finalized

In [7]:
###################################
# Once we finalized our features and model we can train it using the whole training set and then produce prediction for the evaluating dataset
###################################
# Load the evaluation data
eval_data = pd.read_csv("evaluation.csv")
# Transform our data into tfidf vectors
vectorizer = TfidfVectorizer(max_features=100, stop_words=stopwords.words('french'))
train_data = pd.read_csv("train.csv")
y_train = train_data['retweets_count']
X_train = vectorizer.fit_transform(train_data['text'])
train_data = train_data.loc[:,train_columns]
# We fit our model using the training data
reg = GradientBoostingRegressor()
reg.fit(train_data, y_train)
X_val = vectorizer.transform(eval_data['text'])
eval_data_tr = eval_data.loc[:,train_columns]
# Predict the number of retweets for the evaluation dataset
y_pred = reg.predict(eval_data_tr)
y_pred = [int(value) if value >= 0 else 0 for value in y_pred]
# Dump the results into a file that follows the required Kaggle template
with open("gbr_predictions.txt", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "retweets_count"])
    for index, prediction in enumerate(y_pred):
        writer.writerow([str(eval_data['TweetID'].iloc[index]) , str(int(prediction))])


### Catboost

In [9]:
###################################
# Once we finalized our features and model we can train it using the whole training set and then produce prediction for the evaluating dataset
###################################
# Load the evaluation data
eval_data = pd.read_csv("evaluation.csv")
# Transform our data into tfidf vectors
vectorizer = TfidfVectorizer(max_features=100, stop_words=stopwords.words('french'))
train_data = pd.read_csv("train.csv")
y_train = train_data['retweets_count']
X_train = vectorizer.fit_transform(train_data['text'])

train_data = train_data.loc[:,train_columns]
# We fit our model using the training data
model = CatBoostRegressor(
    random_seed=42,
)
categorical_features_indices = []
model.fit(
    train_data, y_train,
    cat_features=categorical_features_indices,
    plot=True
)
X_val = vectorizer.transform(eval_data['text'])
eval_data_tr = eval_data.loc[:,train_columns]
# Predict the number of retweets for the evaluation dataset
y_pred = model.predict(eval_data_tr)
y_pred = [int(value) if value >= 0 else 0 for value in y_pred]
# Dump the results into a file that follows the required Kaggle template
with open("catboost_predictions.txt", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "retweets_count"])
    for index, prediction in enumerate(y_pred):
        writer.writerow([str(eval_data['TweetID'].iloc[index]) , str(int(prediction))])



MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.103494
0:	learn: 227.3084694	total: 19.8ms	remaining: 19.8s
1:	learn: 214.1015269	total: 31.7ms	remaining: 15.8s
2:	learn: 202.3536656	total: 41.8ms	remaining: 13.9s
3:	learn: 192.0202205	total: 52.9ms	remaining: 13.2s
4:	learn: 183.1236626	total: 64.4ms	remaining: 12.8s
5:	learn: 174.4972183	total: 76.1ms	remaining: 12.6s
6:	learn: 167.2204251	total: 88.3ms	remaining: 12.5s
7:	learn: 160.7018324	total: 101ms	remaining: 12.5s
8:	learn: 154.4313337	total: 112ms	remaining: 12.3s
9:	learn: 148.3815698	total: 122ms	remaining: 12.1s
10:	learn: 143.4898758	total: 133ms	remaining: 12s
11:	learn: 139.4225643	total: 144ms	remaining: 11.9s
12:	learn: 134.9648654	total: 154ms	remaining: 11.7s
13:	learn: 131.1704842	total: 165ms	remaining: 11.6s
14:	learn: 127.5485716	total: 176ms	remaining: 11.6s
15:	learn: 123.9356973	total: 189ms	remaining: 11.6s
16:	learn: 120.7580422	total: 201ms	remaining: 11.6s
17:	learn: 118.1206485	total: 212ms	remaining: 11.6s
18:	learn: 115.698726

161:	learn: 57.6980306	total: 1.8s	remaining: 9.31s
162:	learn: 57.6011865	total: 1.81s	remaining: 9.3s
163:	learn: 57.4410389	total: 1.82s	remaining: 9.29s
164:	learn: 57.2944158	total: 1.83s	remaining: 9.28s
165:	learn: 57.1977987	total: 1.84s	remaining: 9.27s
166:	learn: 57.1108058	total: 1.86s	remaining: 9.27s
167:	learn: 56.9330071	total: 1.87s	remaining: 9.26s
168:	learn: 56.8040478	total: 1.88s	remaining: 9.25s
169:	learn: 56.7393698	total: 1.89s	remaining: 9.24s
170:	learn: 56.5818755	total: 1.9s	remaining: 9.23s
171:	learn: 56.5040743	total: 1.92s	remaining: 9.22s
172:	learn: 56.3598442	total: 1.92s	remaining: 9.2s
173:	learn: 56.2094935	total: 1.94s	remaining: 9.19s
174:	learn: 56.0855090	total: 1.95s	remaining: 9.18s
175:	learn: 55.9812128	total: 1.96s	remaining: 9.16s
176:	learn: 55.9277675	total: 1.97s	remaining: 9.15s
177:	learn: 55.8715533	total: 1.98s	remaining: 9.14s
178:	learn: 55.7484590	total: 1.99s	remaining: 9.12s
179:	learn: 55.6219516	total: 2s	remaining: 9.11s


331:	learn: 45.0677054	total: 3.62s	remaining: 7.28s
332:	learn: 45.0463603	total: 3.63s	remaining: 7.28s
333:	learn: 45.0332898	total: 3.64s	remaining: 7.26s
334:	learn: 44.9657457	total: 3.65s	remaining: 7.25s
335:	learn: 44.9439451	total: 3.66s	remaining: 7.24s
336:	learn: 44.9213435	total: 3.67s	remaining: 7.23s
337:	learn: 44.8684566	total: 3.68s	remaining: 7.22s
338:	learn: 44.7711529	total: 3.69s	remaining: 7.21s
339:	learn: 44.7245457	total: 3.71s	remaining: 7.2s
340:	learn: 44.7153633	total: 3.72s	remaining: 7.18s
341:	learn: 44.6731840	total: 3.73s	remaining: 7.17s
342:	learn: 44.6212241	total: 3.74s	remaining: 7.16s
343:	learn: 44.5779885	total: 3.75s	remaining: 7.14s
344:	learn: 44.5126231	total: 3.76s	remaining: 7.13s
345:	learn: 44.4657576	total: 3.77s	remaining: 7.12s
346:	learn: 44.4242085	total: 3.78s	remaining: 7.11s
347:	learn: 44.3903432	total: 3.79s	remaining: 7.1s
348:	learn: 44.3519818	total: 3.8s	remaining: 7.09s
349:	learn: 44.3394519	total: 3.81s	remaining: 7.

502:	learn: 39.0364308	total: 5.43s	remaining: 5.37s
503:	learn: 39.0207314	total: 5.45s	remaining: 5.36s
504:	learn: 39.0145767	total: 5.46s	remaining: 5.35s
505:	learn: 38.9798187	total: 5.47s	remaining: 5.34s
506:	learn: 38.9305850	total: 5.48s	remaining: 5.33s
507:	learn: 38.8840633	total: 5.49s	remaining: 5.32s
508:	learn: 38.8643328	total: 5.5s	remaining: 5.31s
509:	learn: 38.8468196	total: 5.51s	remaining: 5.29s
510:	learn: 38.8238070	total: 5.52s	remaining: 5.28s
511:	learn: 38.7977392	total: 5.53s	remaining: 5.27s
512:	learn: 38.7660409	total: 5.54s	remaining: 5.26s
513:	learn: 38.7472116	total: 5.55s	remaining: 5.25s
514:	learn: 38.7242469	total: 5.56s	remaining: 5.24s
515:	learn: 38.7026812	total: 5.57s	remaining: 5.23s
516:	learn: 38.6839779	total: 5.58s	remaining: 5.22s
517:	learn: 38.6421495	total: 5.59s	remaining: 5.21s
518:	learn: 38.6209332	total: 5.6s	remaining: 5.19s
519:	learn: 38.6020252	total: 5.61s	remaining: 5.18s
520:	learn: 38.5790047	total: 5.62s	remaining: 5

675:	learn: 35.1609187	total: 7.25s	remaining: 3.48s
676:	learn: 35.1453969	total: 7.26s	remaining: 3.46s
677:	learn: 35.1379038	total: 7.27s	remaining: 3.45s
678:	learn: 35.1295658	total: 7.28s	remaining: 3.44s
679:	learn: 35.1175897	total: 7.29s	remaining: 3.43s
680:	learn: 35.1107010	total: 7.3s	remaining: 3.42s
681:	learn: 35.1009570	total: 7.31s	remaining: 3.41s
682:	learn: 35.0839237	total: 7.32s	remaining: 3.4s
683:	learn: 35.0607331	total: 7.33s	remaining: 3.39s
684:	learn: 35.0437084	total: 7.35s	remaining: 3.38s
685:	learn: 35.0189172	total: 7.36s	remaining: 3.37s
686:	learn: 34.9716612	total: 7.37s	remaining: 3.36s
687:	learn: 34.9505548	total: 7.38s	remaining: 3.35s
688:	learn: 34.9285275	total: 7.38s	remaining: 3.33s
689:	learn: 34.9140667	total: 7.39s	remaining: 3.32s
690:	learn: 34.9083112	total: 7.4s	remaining: 3.31s
691:	learn: 34.8932655	total: 7.42s	remaining: 3.3s
692:	learn: 34.8828484	total: 7.42s	remaining: 3.29s
693:	learn: 34.8541197	total: 7.43s	remaining: 3.2

848:	learn: 32.1850593	total: 9.07s	remaining: 1.61s
849:	learn: 32.1775824	total: 9.08s	remaining: 1.6s
850:	learn: 32.1665964	total: 9.09s	remaining: 1.59s
851:	learn: 32.1572649	total: 9.1s	remaining: 1.58s
852:	learn: 32.1463956	total: 9.11s	remaining: 1.57s
853:	learn: 32.1427383	total: 9.12s	remaining: 1.56s
854:	learn: 32.1310498	total: 9.13s	remaining: 1.55s
855:	learn: 32.1175689	total: 9.14s	remaining: 1.54s
856:	learn: 32.1022674	total: 9.16s	remaining: 1.53s
857:	learn: 32.0852500	total: 9.16s	remaining: 1.52s
858:	learn: 32.0757852	total: 9.18s	remaining: 1.51s
859:	learn: 32.0574366	total: 9.19s	remaining: 1.5s
860:	learn: 32.0384891	total: 9.2s	remaining: 1.48s
861:	learn: 32.0293659	total: 9.21s	remaining: 1.47s
862:	learn: 32.0263652	total: 9.22s	remaining: 1.46s
863:	learn: 32.0076752	total: 9.23s	remaining: 1.45s
864:	learn: 31.9967468	total: 9.24s	remaining: 1.44s
865:	learn: 31.9802655	total: 9.25s	remaining: 1.43s
866:	learn: 31.9541071	total: 9.26s	remaining: 1.4